In [7]:
import time
import csv
import random
import pandas as pd
import yeelight
import PySimpleGUI as sg
import pygame.mixer
from mutagen.mp3 import MP3


def load_colors():
    colors = pd.read_csv('Csv file displayed in hexadecimal', header=None)
    return [int(color[1:], 16)
            for index, rows in colors.iterrows()
            for color in rows]


class Light:
    MAX_BRIGHTNESS = 50

    def __init__(self, address):
        self.bulb = yeelight.Bulb(address)

    def turn_on(self):
        self.bulb.turn_on()
        self.bulb.set_brightness(self.MAX_BRIGHTNESS)

    def turn_off(self):
        self.bulb.turn_off()

    def change_color(self, rgb):
        r, g, b = (rgb >> 16) & 0xff, (rgb >> 8) & 0xff, rgb & 0xff
        self.bulb.set_rgb(r, g, b)

    def fade_out(self):
        for bright in range(self.MAX_BRIGHTNESS, 0, -2):
            self.bulb.set_brightness(bright)
            time.sleep(0.1)
        time.sleep(1)

def select_music():
    sg.change_look_and_feel('Light Blue 2')
    terms = { 1: False, 3: False, 5: False, 10: False }
    layout = [[sg.Text('Data'),
               sg.InputText('(file path)', key='filepath'),
               sg.FilesBrowse('Select a file', target='filepath',
                              file_types=(('mp3 file', '.mp3'),))],
              [sg.Text('term')],
              [sg.Radio(f'{sec}sec', key=sec, default=terms[sec], group_id='0')
               for sec in terms],
              [sg.Submit(), sg.Cancel()]
              ]
    window = sg.Window('Charting', layout)
    while True:
        event, values = window.read()
        if event in ('Cancel', sg.WIN_CLOSED):
            return None, 0
        if event == 'Submit':
            break
    window.close()
    filepath = values.get('filepath')
    if filepath == '(file path)':
        filepath = None
    term = [sec for sec in terms if values.get(sec)][0]
    return filepath, term


class Music:
    pygame.mixer.init()

    def __init__(self, filepath):
        self.filepath = filepath

    def length(self):
        return MP3(self.filepath).info.length

    def start(self):
        pygame.mixer.music.load(self.filepath)
        pygame.mixer.music.play()

    def stop(self):
        pygame.mixer.music.stop()


def main():
    # 1. Select a music file
    filepath, term = select_music()
    if filepath is None:
        return

    # 2. Start music
    music = Music(filepath)
    music.start()

    # 2. Turn on the light
    light = Light("IP address"))
    light.turn_on()

    # 3. Change the light's color
    colors = load_colors()
    for _ in range(0, int(music.length()) + 1, term):
        light.change_color(random.choice(colors))
        time.sleep(term)

    # 4. Stop music and turn off the light
    time.sleep(3)
    music.stop()
    light.fade_out()
    light.turn_off()


if __name__ == '__main__':
    main()